In [9]:
# created by zayaan. going to test using a basic neural network for the stuff
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
#we want to create a neural network, so we need to import the keras library
from keras.models import Sequential
import tensorflow as tf

In [10]:
csv_file = "sample_data.csv" 
df = pd.read_csv(csv_file, sep=",")
# individual photometry filters
u = df['u']
g = df['g']
r = df['r']
i = df['i']
z = df['z']

# color indices
u_g = u - g
g_r = g - r
r_i = r - i
i_z = i - z

# u_g = u_g[:-19500]
# g_r = g_r[:-19500]
# r_i = r_i[:-19500]
# i_z = i_z[:-19500]

u_g = u_g[:-19500]
g_r = g_r[:-19500]
r_i = r_i[:-19500]
i_z = i_z[:-19500]

# Effective temperature of star
temperature = df['Teff']

color_filters_df = pd.DataFrame()
color_filters_df['u_g'] = u_g
color_filters_df['g_r'] = g_r
color_filters_df['r_i'] = r_i
color_filters_df['i_z'] = i_z

# Effective temperature of star
color_filters_df['temperature'] = df['Teff']

In [22]:
#the input of the model is the color indices, and the output is the temperature.
#lets perform this with a neural network
#we need to split the data into training and testing data
#we will use 80% of the data for training and 20% for testing
#we will also shuffle the data to avoid any bias
train, test = train_test_split(color_filters_df, test_size=0.2, shuffle=True)
#we need to split the data into input and output
#the input is the color indices
print(color_filters_df['u_g'])


0      1.18387
1      0.87497
2      0.32009
3      0.41901
4      2.15088
        ...   
492    1.13375
493    1.16811
494    1.20487
495    1.10280
496    1.87675
Name: u_g, Length: 497, dtype: float64
